# Importing Required Packages

In [1]:
import pandas as pd
import numpy as np

# Reading the Dataset
- The dataset is stored in a dataframe with the name **data**.
- The data-type of **date** column of the dataset is coverted from **str** to **datetime**.
- The **date** column is then set as the index.

In [2]:
data = pd.read_csv('Index2018.csv')
# print(type(data.date[0]))
data.date = pd.to_datetime(data.date)
# print(type(data.date[0]))
data = data.set_index('date')
# print(len(data.index))
data.head()

,spx,dax,ftse,nikkei
date,,,,
1994-07-01,469.90,2224.95,3445.98,18124.01
1994-10-01,475.27,2225.00,3440.58,18443.44
1994-11-01,474.13,2228.10,3413.77,18485.25
1994-12-01,474.17,2182.06,3372.02,18793.88
1994-01-13,472.47,2142.37,3360.01,18577.26


The summary statistics for the numeric columns of the dataset are given below.

In [3]:
data.describe()

,spx,dax,ftse,nikkei
count,6269.000000,6269.000000,6269.000000,6269.000000
mean,1288.127542,6080.063363,5422.713545,14597.055700
std,487.586473,2754.361032,1145.572428,4043.122953
min,438.920000,1911.700000,2876.600000,7054.980000
25%,990.671905,4069.350000,4486.100000,10709.290000
50%,1233.420000,5773.340000,5662.430000,15028.170000
75%,1459.987747,7443.070000,6304.250000,17860.470000
max,2872.867839,13559.600000,7778.637689,24124.150000


# Converting the data to time-series format.
Since the interval between periods in a time-series data must be equal, we set the frequency as **b** (Business Days). On adding this, we get a few more tuples in the dataset with missing values for each of the indices. 

In [4]:
data = data.asfreq('b')
data.isna().sum()

spx       686
dax       686
ftse      686
nikkei    686
dtype: int64

# Filling the missing values
The missing values for each column is filled using the **ffill** (front-fill) method of the **fillna** function. With this function, the missing value is filled with the next available value.

In [5]:
data.spx = data.spx.fillna(method='ffill')
data.dax = data.dax.fillna(method='ffill')
data.ftse = data.ftse.fillna(method='ffill')
data.nikkei = data.nikkei.fillna(method='ffill')

data.isna().sum()

spx       0
dax       0
ftse      0
nikkei    0
dtype: int64

# Calculating Retruns for the Prices
The returns for a index can be calculated using the following formula - <br>
$Returns(t) = Prices(t) - Prices(t-1)$ <br>
Here $t$ is the time period in question. <br><br>

On calculating the returns for each index, the data is concatenated to the dataframe itself.

In [6]:
data['spx_ret'] = data.spx.pct_change(1).mul(100)
data['dax_ret'] = data.dax.pct_change(1).mul(100)
data['ftse_ret'] = data.ftse.pct_change(1).mul(100)
data['nikkei_ret'] = data.nikkei.pct_change(1).mul(100)

data.head()

,spx,dax,ftse,nikkei,spx_ret,dax_ret,ftse_ret,nikkei_ret
date,,,,,,,,
1994-07-01,469.90,2224.95,3445.98,18124.01,NaN,NaN,NaN,NaN
1994-07-04,450.88,2184.89,3129.00,19890.98,-4.047670,-1.800490,-9.198544,9.749333
1994-07-05,450.88,2184.89,3129.00,19890.98,0.000000,0.000000,0.000000,0.000000
1994-07-06,458.21,2131.80,3004.80,21042.71,1.625710,-2.429871,-3.969319,5.790212
1994-07-07,448.38,2049.10,2964.40,20620.02,-2.145305,-3.879351,-1.344515,-2.008724


# Calculating Normalized Prices
The Normalized prices of an index can be used to accurately compare the growth and fall of each market. <br>
It can be calculated using the following formula - <br>
$Normalized Prices(t) = (Prices(t)/Bench Mark) * 100$ <br>
Here $t$ is the time period in question. <br>
And, $Bench Mark$ is the baseline used for each marke index. For this analysis the first value of each index is chosen <br><br>

On calculating the noralized prices, the data is concatenated to the dataframe itself.

In [7]:
spx_benchmark = data.spx[0]
data['spx_norm'] = data.spx.div(spx_benchmark).mul(100)

dax_benchmark = data.dax[0]
data['dax_norm'] = data.dax.div(dax_benchmark).mul(100)

ftse_benchmark = data.ftse[0]
data['ftse_norm'] = data.ftse.div(ftse_benchmark).mul(100)

nikkei_benchmark = data.nikkei[0]
data['nikkei_norm'] = data.nikkei.div(nikkei_benchmark).mul(100)

data.head()

,spx,dax,ftse,nikkei,spx_ret,dax_ret,ftse_ret,nikkei_ret,spx_norm,dax_norm,ftse_norm,nikkei_norm
date,,,,,,,,,,,,
1994-07-01,469.90,2224.95,3445.98,18124.01,NaN,NaN,NaN,NaN,100.000000,100.000000,100.000000,100.000000
1994-07-04,450.88,2184.89,3129.00,19890.98,-4.047670,-1.800490,-9.198544,9.749333,95.952330,98.199510,90.801456,109.749333
1994-07-05,450.88,2184.89,3129.00,19890.98,0.000000,0.000000,0.000000,0.000000,95.952330,98.199510,90.801456,109.749333
1994-07-06,458.21,2131.80,3004.80,21042.71,1.625710,-2.429871,-3.969319,5.790212,97.512237,95.813389,87.197256,116.104052
1994-07-07,448.38,2049.10,2964.40,20620.02,-2.145305,-3.879351,-1.344515,-2.008724,95.420302,92.096452,86.024875,113.771842


# Export DataFrame
The preprocessed data is exported as a **.csv** file.

In [8]:
data.to_csv('data.csv')